In [6]:
!pip install -r requirements.txt --ignore-installed

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 1.6 MB/s eta 0:00:08
     - -------------------------------------- 0.4/12.8 MB 3.7 MB/s eta 0:00:04
     -- ------------------------------------- 0.9/12.8 MB 5.6 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 7.2 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/12.8 MB 8.5 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 9.5 MB/s eta 0:00:02
     ---------- ----------------------------- 3.5/12.8 MB 10.1 MB/s eta 0:00:01
     ------------ --------------------------- 4.1/12.8 MB 10.4 MB/s eta 0:00:01
     -------------- ------------------------- 4.8/12.8 MB 10.8 MB/s eta 0:00:01
     ---------------- ----------------------- 5.4/12.8 MB 11.1 MB/s eta 0:00:01
     ------------------ --------------------- 6.1/12.8 MB 11.4

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '\\colabtools\\dist\\google-colab-1.0.0.tar.gz'



In [5]:
import numpy as np, scipy as sc

import gym
from gym.utils import EzPickle

import cProfile  as cProfile

from my_envs.soil_system import SoilGreenHouse

import torch
 
from sac_torch import AgentSAC
import gym
# from SAC import SAC_Agent
from ReplayBuffer import RandomBuffer, device
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os, shutil
# import argparse
from Adapter import *
#
import gym
import numpy as np
from utils import plotLearning

import glob
from PIL import Image

# from utils import show_video, convert_gif
# from gym.wrappers.monitoring.video_recorder import VideoRecorder


from base64 import b64encode

ModuleNotFoundError: No module named 'scipy'

## Выкладки

Концепция регулировки
* Вентиль 1: **Раствор с высокой концентрацией**: 
> Раствор А может содержать высокую концентрацию азота, фосфора и калия (NPK) для быстрого и интенсивного роста растений. Например, его состав может быть 20-10-10, что означает, что концентрация азота составляет 20%, фосфора - 10%, а калия - 10%, кислотность pH = 6
* Вентиль 2: **Раствор с низкой концентрацией**: 
> Раствор В может содержать низкую концентрацию NPK для поддержания стабильного роста растений. Например, его состав может быть 5-5-5, что означает, что концентрация азота, фосфора и калия составляет 5% каждый, кислотность pH = 6
* Вентиль 3: Чистая вода, кислотность pH = 7
> В чистой воде концентрация ионов [OH-] и [H+] является равной и составляет 10^-7 Моль/литр. 

> Показатели элементов N (азот), P (фосфор) и K (калий) в чистой воде обычно очень низкие и близки к нулю. Они могут присутствовать в следующих количествах:

> - Азот (N): обычно присутствует в виде аммиака (NH3) или нитратов (NO3-) в очень низких концентрациях, обычно менее 1 мг/литр.
> - Фосфор (P): присутствует в виде фосфатов (PO4^3-) в очень низких концентрациях, обычно менее 0,1 мг/литр.
> - Калий (K): присутствует в виде ионов K+ в очень низких концентрациях, обычно менее 1 мг/литр.

**Объем бака с раствором для конкретного горшка с субстратом: V = 8.5  литров**

### **Для симуляции изменения среды со временем, необходимо расчитывать изменения следующих показателей среды**:
* $T_{soil}$ - температура среды
* $\phi_{soil}$ - влажность  среды
* $\text{pH}_{soil}$ - кислотность среды
* $\text{EC/TDS}_{soil}$ - минерализация среды
* $\text{N}_{soil}$,$\text{P}_{soil}$,$\text{K}_{soil}$  - NPK показатели

#### $T_{soil}$ - температура среды

Расчет температуры почвы (средняя температура по горщку) :
$$
T(t) = \frac{1}{2 \cdot \pi \cdot \text{R} \cdot \text{h}} \int_{0}^{\text{R}} \int_{0}^{2 \cdot \pi} [T_{air} + (T(t-1) - T_{air}) * \text{sin}(\theta) * \text{r} \cdot \text{exp}(-\lambda^2 * dt)]
$$
где 
> $\text{R}$ - радиус цилиндрического горшка;

> $\text{h}$ - высота цилиндрического горшка;

> $T_{air} $ - температура воздуха;

> $T(t)$ - температура почвы в момент времени $t$;

> $\lambda$ - Коэффициент теплопроводности;

```python
def temp_soil(
    dt:float,T_soil_0:float,
    R:float=0.35,h:float=0.35,
    T_air:float=299,
    _lambda:float=np.random.uniform(0.8,1.16)) -> np.ndarray(shape=(1,),dtype=np.float64):
    """
        Данный метод используется для расчета 
        точного значения средней температуры в 
        цилиндрическом горшке с радиусом R и высотой h. 
        Формула учитывает разницу в начальной температуре 
        Tₐ и конечной температуре Tᵢ, 
        а также время t, прошедшее
        с момента начала нагрева или охлаждения.

        Args:
            dt (float): Time. Measured in sec.
            T_air (float): Soi temperature (old). Measured in K;
            R (float): Radius of the cylindrical pot. Measured in meters. Default to 0.35; 
            h (float): Height of the cylindrical pot. Measured in meters. Default to 0.35; 
            T_air (float): Air temperature. Measured in K. Defaults to 26+273;
            _lambda(float): Coefficient of thermal conductivity. Measured in W/(m*K). Defaults to np.random.uniform(0.8,1.16);
        Result:
            np.ndarray(shape=(1,),dtype=np.float64) Calculated temperature of soil in the pot and an estimate of the error.
    """
         
    f = lambda r, theta: T_air + (T_soil_0 - T_air) * np.sin(theta) * r/np.exp(_lambda**2 * dt)
    mu,sigma = 1/(2*np.pi*R*h)*np. asarray(integrate.dblquad(f, 0, R, 0, 2*np.pi))
    return np.random.normal(mu, sigma, 1)
```

#### $\phi_{soil}$ - влажность  среды

$$
\frac{dH}{dt} =  P - E
$$ 
где 
* $P$ - поступление влаги в почву (растение в гидропонной системе через корни впитываемой через корни со скоростью V); 
* $E$ - испарения влаги из почвы (известна влажность и температура окружающего воздуха)

В первую очередь необходимо расчитать концентрацию каждого из 3 растворов в горшке и 4 вентиля: 3 для растворов + сброс воды. Необходимо учитывать скорость всасывания растением на определенной стадии питательного раствора.
В первом приближении предположим :
* Скорость всасывания раствора $V = k * (RH - AH)$ где 
> $RH$ - относительная влажность вочвы

> $AH$ - абсолютная влажность воздуха

> $k$ -  коэффициент пропорциональности, зависящий от свойств почвы и растения.Например, для чернозема и томатов коэффициент пропорциональности может составлять примерно от *0.5* до *2.0*, где более низкие значения указывают на более низкую эффективность питательных веществ в почве для роста и развития томатов, а более высокие значения указывают на более высокую эффективность. Однако, следует отметить, что эти значения являются приблизительными и могут изменяться в зависимости от многих факторов. Поэтому рекомендуется провести конкретные эксперименты или обратиться к специализированным исследованиям для получения более точных численных значений коэффициента пропорциональности между черноземом и томатами.
* Скорость подачи растворов $V_{in} = 15 \frac{мм^{3}}{час}$
* Скорость отвода растворов $V_{in} = 55 \frac{мм^{3}}{час}$
Учитывая что скорость всасывания зависит напрямую от показателей датчиков, то сначала изменяется влажность почвы, затем в зависимости от расчитанной влажности выставляется новый коэффициент и новая скорость употребления раствора $V$

* Формула для расчета поступления влаги $P$ будет следующей:

$$ P = V \cdot A $$

где $V$ - скорость всасывания влаги через корни растения, $A$ - площадь поверхности корней, через которую происходит всасывание.

* Формула для расчета испарения влаги $E$ будет зависеть от влажности и температуры окружающего воздуха. Одна из самых распространенных формул для расчета испарения - формула Пенмана-Монтефиори:

$$ E = K \cdot (e_s - e_a) + \frac{c_p \cdot \Delta R_n}{\lambda} $$

где 
* $K$ - коэффициент, учитывающий условия испарения (включая скорость ветра и солнечную радиацию), 
* $e_s$ - насыщенное парциальное давление воды при данной температуре, 
* $e_a$ - фактическое парциальное давление воды, 
* $c_p$ - удельная теплоемкость воздуха при постоянном давлении, 
* $R_n$ - чистый радиационный баланс (разность между входящей и исходящей радиацией), 
* $\lambda$ - удельная теплота испарения воды.

Для упрощения формулы Пенмана-Монтефиори в тепличных условиях и при принятии default значения солнечной радиации, можно убрать слагаемое $\frac{c_p \cdot \Delta R_n}{\lambda}$ из формулы. Таким образом, упрощенная формула будет выглядеть следующим образом:

$$ E = K \cdot \Delta (e_s - e_a) $$

Для расчета значений $e_s$, $e_a$ и $c_p$ для воздуха в диапазоне от 5 до 35 градусов Цельсия можно использовать следующие формулы:

1) Формула для расчета насыщенного парциального давления воды $e_s$ при данной температуре:

$$ e_s = 0.6108 \cdot \exp\left(\frac{17.27 \cdot T}{T + 237.3}\right) $$

где $T$ - температура в градусах Цельсия.

2) Формула для расчета фактического парциального давления воды $e_a$ при данной температуре и относительной влажности $RH$:

$$ e_a = RH \cdot e_s $$

3) Значение удельной теплоемкости воздуха $c_p$ при постоянном давлении можно принять равным 1005 Дж/(кг·К).

Для удельной теплоты испарения воды $\lambda$ в тепличных условиях можно принять значение около 2.45 МДж/кг.

При использовании упрощенной формулы Пенмана-Монтефиори и приведенных выше формул для расчета $e_s$, $e_a$ и $c_p$, необходимо учесть, что эти значения могут быть приближенными и не учитывать все факторы, влияющие на испарение влаги.


Итоговая формула расчета влажности:
$$
\frac{dH}{dt} =  k \cdot (RH - AH) \cdot A - (K \cdot (e_s - e_a) + \frac{c_p \cdot (R_{in} - R_{out})}{\lambda})
$$

Разностное уравнение:

$$
\Delta H = \Delta t \cdot \left( k \cdot (RH - AH) \cdot A - \left( K \cdot (e_s - e_a) + \frac{c_p \cdot (R_{in} - R_{out})}{\lambda} \right) \right)
$$

В разностном уравнении $\Delta H$ представляет изменение влажности в течение временного шага $\Delta t$.

Алгоритм:

1. Задайте начальное значение $H$ (например, $H_0$) и начальное время $t_0$.

2. Задайте временной шаг $\Delta t$ (например, $0.1$ дней).

3. На каждом временном шаге $n$ вычислите:

   $$H_{n+1} = H_n + \Delta t \cdot \left[k \cdot (RH - AH) \cdot A - \left(K \cdot (e_s - e_a) + \frac{c_p \cdot (R_{in} - R_{out})}{\lambda}\right)\right]$$

   где $H_n$ - значение влажности на предыдущем временном шаге.

4. Увеличивайте время $t$ на $\Delta t$ и повторяйте шаг 3, пока не достигнете нужного момента времени.

Это численный метод, который позволит  интегрировать уравнение во времени, при условии, что  есть начальные значения и заданные параметры.

Обратите внимание, что для точных результатов, возможно, потребуется использовать более сложные методы численного интегрирования, такие как методы Рунге-Кутты, особенно если есть изменяющиеся параметры во времени.

```python
def calculate_diff_eq(k, RH, AH, A, K, e_s, e_a, c_p, lmbda, R_in, R_out, dt):
    # Вычисляем изменение влажности по формуле дифференциального уравнения
    delta_H = dt * (k * (RH - AH) * A - (K * (e_s - e_a) + (c_p * (R_in - R_out)) / lmbda))
    return delta_H
```

#### $\text{pH}_{soil}$ - кислотность среды

Расчет $pH$:
$$
pH(t) = pH_0 + log_{10}(\frac{[H+](t)}{[OH-](t)}) 
$$, где 
> $pH_0$ - начальное значение $pH$,

> $[H+](t)$ - концентрация ионов водорода в момент времени t, 
  
> $[OH-](t)$ - концентрация ионов гидроксида в момент времени t.

> Общая концентрация ионов `H+`: 
$$
\text{H+} = \frac{N_1 \cdot 10^{-pH_{A}} + N_2 \cdot 10^{-pH_{B}} + N_3 \cdot 10^{-pH_{water}}}{N_1 + N_2 + N_3}
$$
> Общая концентрация ионов `OH-`:
$$
\text{OH-} = \frac{N_1 \cdot 10^{-(pH_{A} + 20)} + N_2 \cdot 10^{-(pH_{B} + 20)} + N_3 \cdot 10^{-(pH_{\text{water}} + 20)}}{N_1 + N_2 + N_3}
$$
где  $N_1$, $N_2$ и $N_3$ - объемы трех растворов `A`, `B` и чистой воды соответственно.

#### $\text{EC/TDS}_{soil}$ - минерализация среды

Расчет $\text{EC/TDS}$:   
$$
\text{EC/TDS}(t) = \text{EC}_0 + ∑_i ([C_i](t) \cdot z_i \cdot f_i)
$$
В качестве $C_i$  будем считать концентрации элементов  `N`,`P`,`K`. *Заряды* соответствующих ионов {-3} {-3} {+1}
**Необходимо предварительно расчитать концентрации элементов**.

#### $\text{N}_{soil}$,$\text{P}_{soil}$,$\text{K}_{soil}$  - NPK показатели

Расчет параметров NPK: 
$$ 
N(t) = N_0 +  ([C_i](t) \cdot f_i);    
P(t) = P_0 +  ([C_i](t) \cdot f_i); 
K(t) = K_0 +  ([C_i](t) \cdot f_i)  
$$  
где $N_0$, $P_0$, $K_0$ - начальные значения `N`, `P`, `K`, $[С_i](t)$ - концентрация `i`-го иона в момент времени `t`, $f_i$ - коэффициент фактора, учитывающего влияние на `N`, `P`, `K`.

### Финальные формулы используемые для расчета

**Формулы для расчета состояния почвы в следующий момент времени** (основные формулы):
1. Расчет температуры почвы :
$$
T(t) = \frac{1}{2 \cdot \pi \cdot R \cdot h} \int_{0}^{R} \int_{0}^{2 \cdot \pi} [T_{air} + (T(t-1) - T_{air}) * \text{sin}(\theta) * \frac{\text{r}}{\text{exp}(\lambda^2 * dt)}]
$$

2. Расчет влажности:
$$
 \phi(t) = [V \cdot A   - K \cdot (e_s - e_a)   - \frac{c_p \cdot (R_{in} - R_{out})}{\lambda}] \cdot  (t - t_0) + \phi(t_0)
$$

3. Расчет $pH$:
$$
pH(t) = pH_0 + log_{10}(\frac{[H+](t)}{[OH-](t)}) 
$$
4. Расчет $\text{EC/TDS}$:   
$$
\text{EC/TDS}(t) = \text{EC}_0 + ∑_i ([C_i](t) \cdot z_i \cdot f_i)
$$

5. Расчет параметров NPK: 
$$ 
N(t) = N_0 + ∑_i ([C_i](t) \cdot f_i); 
$$
$$
P(t) = P_0 + ∑_i ([C_i](t) \cdot f_i); 
$$
$$
K(t) = K_0 + ∑_i ([C_i](t) \cdot f_i)  
$$  

### **Вспомогательные формулы для расчета концентраций тех или иных элементов для расчета**




Необходимо расчитать концентрации следующих ионов:
* Расчет концентрации [`H+`]: 
* Расчет концентрации [`OH-`]: 
* Расчет концентрации ионов `N`, `P`, `K`:



*Условие 1*: объем раствора в баке необходимо держать постоянным,  то есть необходима слаженная работа всех 3 + 1 вентилей контроля раствора.

*Условие баланса раствора в баке*:
$$
V_1+V_2+V_3 = V_{out} + V
$$ где 
> $V_1$,$V_2$,$V_3$ - скорости подачи из трех вентилей с растворами;
> $V_{out}$  - скорость отвода смеси из бака;
> $V$ - скорость усваивания раствора почвой (корректируется в каждый момент замера)

*Условие 2*: Замеры происходят через $\delta t = 1\text{ч}$

Объема раствора i- за время $t$: $$ V_i = U \cdot a_{i} \cdot t $$ где $U = 15 \frac{мм^{3}}{час}$ - стандартная скорость поступления, $a_{i} \in [0,1] $ - степень открытия винта

Тогда концентрации `i`-ого иона (N,P,K):
> Расчет N: $C_{N}(t) = \sum^{A,B,Water}_{i=A} alpha^{N}_{i} * V_{i} [\frac{\text{грамм}}{литр}] =N(t) \cdot 0.071 [\frac{\text{моль}}{литр}]$ где $alpha^{N}_{i}$ - коэффициент содержания `N` в `i`-ом  растворе (20% и 5% в растворах A и B соответственно, 0% в чистой воде)

> Расчет P: $C_{P}(t) = \sum^{A,B,Water}_{i=A} alpha^{P}_{i} * V_{i} [\frac{\text{грамм}}{литр}] =K(t) \cdot 0.032 [\frac{\text{моль}}{литр}]$ - коэффициент содержания `P` в `i`-ом  растворе (10% и 5% в растворах A и B соответственно,0% в чистой воде)

> Расчет K: $C_{K}(t) = \sum^{A,B,Water}_{i=A} alpha^{K}_{i} * V_{i} [\frac{\text{грамм}}{литр}] =K(t) \cdot 0.026 [\frac{\text{моль}}{литр}]$- коэффициент содержания `K` в `i`-ом  растворе (10% и 5% в растворах A и B соответственно,0% в чистой воде)

*Примечание*: в чистой воде соответствующие концентрации равны нулю.

### Заготовка для модели среды

Простая сверточная автокодировщик (Convolutional Autoencoder) для задачи преобразования входного вектора размерности (8,) в выходной вектор размерности (7,) может выглядеть следующим образом. В данном примере мы будем использовать библиотеку PyTorch для создания нейронной сети.

```python
import torch
import torch.nn as nn

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        # Кодировщик (Encoder)
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),  # Сверточный слой 1
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),  # Сверточный слой 2
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2)  # Пулинг слой
        )
        
        # Декодировщик (Decoder)
        self.decoder = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1),  # Сверточный слой 3
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1),  # Сверточный слой 4
            nn.ReLU(inplace=True),
            nn.ConvTranspose1d(in_channels=8, out_channels=7, kernel_size=2, stride=2)  # Транспонированный сверточный слой
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Создаем экземпляр модели
model = ConvAutoencoder()
```

Обратите внимание, что это простая архитектура, и ее можно дополнительно настраивать и улучшать в зависимости от конкретных требований задачи. Вы можете использовать различные функции активации, функции потерь и оптимизаторы в зависимости от ваших потребностей.

Пример использования модели для преобразования входного вектора и расчета выходного вектора:

```python
# Пример входного вектора
input_vector = torch.randn(1, 8, 10)  # Пример входного вектора (пакет размером 1, 8 признаков, 10 временных шагов)

# Прямой проход (forward pass) через модель
output_vector = model(input_vector)

# Распечатываем выходной вектор
print(output_vector)
```

Обратите внимание, что этот код представляет только основу для построения автокодировщика, и для его обучения и оценки потребуется дополнительный код для подготовки данных, выбора функции потерь и т. д.

Для каждой из указанных моделей создадим простые примеры кода, где каждая модель будет обучаться на временных рядах размерности (N, 8) и предсказывать вектор состояния `h` размерности (7,). Обратите внимание, что эти примеры служат базовой основой и могут потребовать дополнительных настроек и оптимизаций для вашей конкретной задачи.

### MSET
```python
import torch
import torch.nn as nn

class MSET(nn.Module):
    def __init__(self):
        super(MSET, self).__init__()
        self.linear = nn.Linear(8, 7)

    def forward(self, x):
        return self.linear(x)

model_mset = MSET()
```

### LSTM-AE
```python
import torch
import torch.nn as nn

class LSTMAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMAutoencoder, self).__init__()
        self.encoder = nn.LSTM(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        encoded, _ = self.encoder(x.unsqueeze(0))
        decoded = self.decoder(encoded.squeeze(0))
        return decoded

hidden_size = 7
model_lstm_ae = LSTMAutoencoder(input_size=8, hidden_size=hidden_size)
```
### Vanilla LSTM
```python
import torch
import torch.nn as nn

class VanillaLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        output = self.fc(hidden[-1])
        return output

hidden_size = 7
model_vanilla_lstm = VanillaLSTM(input_size=8, hidden_size=hidden_size, output_size=7)
```


### Vanilla AE
```python
import torch
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(True),
            nn.Linear(16, 8),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(True),
            nn.Linear(16, 8),
            nn.ReLU(True)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model_vanilla_ae = Autoencoder()
```

### Isolation Forest
```python
from sklearn.ensemble import IsolationForest

model_isolation_forest = IsolationForest()
```

Обратите внимание, что некоторые из перечисленных вами методов, такие как T-squared+Q (PCA-based), MSCRED, LSTM-VAE и T-squared, представляют собой более сложные модели, требующие дополнительных библиотек и настроек, и не могут быть реализованы в нескольких строках кода. Для подобных моделей лучше обратиться к специализированным библиотекам и ресурсам.

https://github.com/waico/SKAB/tree/master/algorithms

Для модификации данного кода так, чтобы он принимал вектор размерностью (8,) на входе и моделировал вектор состояния h размерностью (7,), нужно внести следующие изменения:

1. Изменить размерности входного и выходного слоя модели.

2. Изменить атрибуты `input_dim` и `latent_dim`.

3. Изменить метод `_build_model` для соответствующей архитектуры.

4. Изменить метод `predict`, чтобы он возвращал вектор состояния h размерностью (7,).

Вот обновленный код:

```python
import numpy as np
from tensorflow.keras.layers import Input, Dense, Lambda, LSTM, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import losses
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

class LSTM_VAE:
    """
    A reconstruction LSTM variational autoencoder model to detect anomalies in timeseries data using reconstruction error as an anomaly score.

    Parameters
    ----------
    TenserFlow_backend : bool, optional
        Flag to specify whether to use TensorFlow backend (default is False).

    Attributes
    ----------
    None

    Examples
    -------
    >>> from LSTM_VAE import LSTM_VAE
    >>> model = LSTM_VAE()
    >>> model.fit(train_data)
    >>> predictions = model.predict(test_data)
    """

    def __init__(self):
        pass

    def _build_model(self,
                     input_dim,
                     intermediate_dim,
                     latent_dim):

        self._Random(0)

        x = Input(shape=(input_dim,))

        h = Dense(intermediate_dim)(x)

        self.z_mean = Dense(latent_dim)(h)
        self.z_log_sigma = Dense(latent_dim)(h)

        z = Lambda(self.sampling, output_shape=(latent_dim,))([self.z_mean, self.z_log_sigma])

        decoder_h = Dense(intermediate_dim)
        decoder_mean = Dense(input_dim)

        h_decoded = decoder_h(z)

        x_decoded_mean = decoder_mean(h_decoded)

        vae = Model(x, x_decoded_mean)

        encoder = Model(x, self.z_mean)

        decoder_input = Input(shape=(latent_dim,))

        _h_decoded = decoder_h(decoder_input)

        _x_decoded_mean = decoder_mean(_h_decoded)
        generator = Model(decoder_input, _x_decoded_mean)

        vae.compile(optimizer='rmsprop', loss=self.vae_loss)

        return vae, encoder, generator

    def _Random(self, seed_value):

        import os
        os.environ['PYTHONHASHSEED'] = str(seed_value)

        import random
        random.seed(seed_value)

        import numpy as np
        np.random.seed(seed_value)

        import tensorflow as tf
        tf.random.set_seed(seed_value)

    def sampling(self, args):
        """
        Sample from the latent space using the reparameterization trick.

        Parameters
        ----------
        args : list
            List of tensors [z_mean, z_log_sigma].

        Returns
        -------
        z : tensorflow.Tensor
            Sampled point in the latent space.
        """
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(self.latent_dim,), mean=0., stddev=self.epsilon_std)
        return z_mean + z_log_sigma * epsilon

    def vae_loss(self, x, x_decoded_mean):
        """
        Calculate the VAE loss.

        Parameters
        ----------
        x : tensorflow.Tensor
            Input data.
        x_decoded_mean : tensorflow.Tensor
            Decoded output data.

        Returns
        -------
        loss : tensorflow.Tensor
            VAE loss value.
        """
        mse = losses.MeanSquaredError()
        xent_loss = mse(x, x_decoded_mean)
        kl_loss = -0.5 * K.mean(1 + self.z_log_sigma - K.square(self.z_mean) - K.exp(self.z_log_sigma))
        loss = xent_loss + kl_loss
        return loss

    def fit(self, data, epochs=20, validation_split=0.1, BATCH_SIZE=1, early_stopping=True):
        """
        Train the LSTM variational autoencoder model on the provided data.

        Parameters
        ----------
        data : numpy.ndarray
            Input data for training.
        epochs : int, optional
            Number of training epochs (default is 20).
        validation_split : float, optional
            Fraction of the training data to be used as validation data (default is 0.1).
        BATCH_SIZE : int, optional
            Batch size for training (default is 1).
        early_stopping : bool, optional
            Whether to use early stopping during training (default is True).
        """

        self.input_dim = data.shape[-1]
        self.latent_dim = 7  # Dimensionality of the latent space
        self.epsilon_std = 1.
        self.intermediate_dim = 32

        self.model, self.enc, self.gen = self._build_model(self.input_dim,
                                                           intermediate_dim=self.intermediate_dim,
                                                           latent_dim=self.latent_dim)

        callbacks = []
        if early_stopping:
            callbacks.append(EarlyStopping(monitor="val_loss", patience=5, mode="min", verbose=0))

        self.model.fit(
            data,
            data,
            epochs=epochs,
            batch_size=BATCH_SIZE,
            validation_split=validation_split,
            verbose=0,
            callbacks=callbacks)

    def predict(self, data):
        """
        Generate predictions using the trained LSTM variational autoencoder model.

        Parameters
        ----------
        data : numpy.ndarray
            Input data for making predictions.

        Returns
        -------
        predictions : numpy.ndarray
            The reconstructed output predictions.
        """

        return self.enc.predict(data)  # Return the encoded representation of the input data
```

Теперь этот класс будет принимать входные данные размерностью (8,) и генерировать вектор состояния h размерностью (7,).

 $$ \text{EC} [\mu\text{S/cm}] = \sum C_i \cdot z_i\ [\text{микросименс на сантиметр}, \mu\text{S/cm}]$$

Давайте пересчитаем формулы с учетом расчета концентрации и объемов для каждого иона и вставим их в качестве вспомогательных формул для каждого иона:

### Основные формулы:

1. **Расчет pH итоговой смеси**:

   $$pH = -\log_{10}(H^+) [\text{безразмерная}]$$

2. **Расчет Электропроводности (EC)**:

    $$ \text{EC} [\mu\text{S/cm}] = \sum C_i \cdot z_i\ [\text{микросименс на сантиметр}, \mu\text{S/cm}]$$

3. **Расчет концентрации азота (N)**:

   $$ \text{Концентрация N}\ [\text{mg/kg}] = ([NO_3^-]_1 \cdot \alpha_{N_{NO_3^-}} \cdot 14) + ([N^+]_2 \cdot \alpha_{N_{NH_4^+}} \cdot 14)\ [\text{миллиграмм на килограмм},\text{mg/kg}]$$

4. **Расчет концентрации фосфора (P)**:

   $$ \text{Концентрация P}\ [\text{mg/kg}] = ([H_2PO_4^-]_1 \cdot \alpha_{P_{H_2PO_4^-}} \cdot 31)\ [\text{миллиграмм на килограмм},\text{mg/kg}]$$

5. **Расчет концентрации калия (K)**:

   $$ \text{Концентрация  K}\ [\text{mg/kg}] = ([K^+]_1 \cdot \alpha_{K_{K^+}} \cdot 39)\ [\text{миллиграмм на килограмм},\text{mg/kg}]$$

### Вспомогательные формулы для каждого иона:

6. **Расчет концентрации водородных ионов ([H+])**:

   $$C_{H^+} = [H^+]_1 \cdot V_1\ ,[моль/литр, M]$$

7. **Расчет концентрации нитратных ионов ([NO3^-])**:

   $$C_{NO_3^-} = [NO_3^-]_1 \cdot V_1\ ,[моль/литр, M]$$

8. **Расчет концентрации аммонийных ионов ([N^+])**:

   $$C_{N^+} = [N^+]_2 \cdot V_2\ ,[моль/литр, M]$$

9. **Расчет концентрации дигидрофосфатных ионов ([H2PO4^-])**:

   $$C_{H2PO4^-} = [H2PO4^-]_1 \cdot V_1\ ,[моль/литр, M]$$

10. **Расчет концентрации калиевых ионов ([K+])**:

    $$C_{K^+} = [K^+]_1 \cdot V_1\ ,[моль/литр, M]$$

11. **Расчет концентрации магниевых ионов ([Mg^{2+}])**:

    $$C_{Mg^{2+}} = [Mg^{2+}]_2 \cdot V_2\ ,[моль/литр, M]$$

12. **Расчет концентрации кальциевых ионов ([Ca^{2+}])**:

    $$C_{Ca^{2+}} = [Ca^{2+}]_2 \cdot V_2\ ,[моль/литр, M]$$

13. **Расчет концентрации гидроксидных ионов ([OH^-])**:

    $$C_{OH^-} = [OH^-]_3 \cdot V_3\ ,[моль/литр, M]$$

Где:
- $(\alpha_{N_{NO_3^-}}, \alpha_{N_{NH_4^+}}, \alpha_{P_{H_2PO_4^-}}, \alpha_{K_{K^+}})$ - массовые доли азота, фосфора и калия в соответствующих соединениях (в данном случае, они равны 1).
- $([NO_3^-]_1, [N^+]_2, [H_2PO_4^-]_1, [K^+]_1)$ - концентрации соответствующих ионов в моль/литр.
- $(z_i)$ - заряд каждого иона (\(z_i\) для каждого иона перечислен в скобках в списке ионов).
- $(V_j)$ - объем раствора, содержащего данный ион, в литрах (для соответствующего раствора).

Для расчета концентраций соответствующих ионов в моль/литр (\([NO_3^-]_1, [N^+]_2, [H_2PO_4^-]_1, [K^+]_1\)) и зарядов каждого иона (\(z_i\)), мы будем использовать информацию о составе каждого раствора. В данном случае, у нас есть три раствора с указанными химическими составами и объемами \(V_1, V_2, V_3\).

Для каждого раствора, мы можем вычислить концентрации и заряды ионов следующим образом:

1. **Расчет концентраций ионов (\([i]_j\)) в моль/литр**:

   Концентрация иона $[i]_j$ в моль/литр для каждого раствора $j$ можно рассчитать, зная массовую долю соответствующего элемента в растворе и объем раствора:

   $$[i]_j = \frac{{\alpha_i \cdot Массовая\ доля\ элемента\ i \cdot V_j}}{{Молекулярная\ масса\ иона\ i}}$$

   Где:
   - $\alpha_i$- массовая доля элемента $i$ в соединении.
   - $Массовая\ доля\ элемента\ i$ - массовая доля элемента $i$ в растворе (например, для азотной кислоты, аммиачной соли и т. д.).
   - $V_j$ - объем раствора $j$ в литрах.
   - $Молекулярная\ масса\ иона\ i$ - молекулярная масса иона $i$.

2. **Расчет зарядов каждого иона (\(z_i\))**:

   Заряд иона ($z_i$) для каждого иона $i$ известен из химических формул соединений. Например, для нитратных ионов [$NO_3^-$] и аммонийных ионов [$N^+$], заряды равны -1 и +1 соответственно.

Теперь, когда у нас есть формулы для расчета концентраций ионов и известны заряды каждого иона, мы можем использовать их в формулах для расчета концентрации водородных ионов, суммарной концентрации ионов и других параметров.

![Alt text](image.png)


В этих формулах $t$ - время, $R_i$ - скорость изменения соответствующего параметра в конкретной добавке. Формулы позволяют учитывать изменение параметров в питательном растворе с течением времени под воздействием добавленных смесей.

##  Практика

```
    mixtures_growth = {
        0:
            Состав (на 100 литров воды):
                Азот (N): 20 г (200 000 ppm)
                Фосфор (P): 10 г (100 000 ppm)
                Калий (K): 20 г (200 000 ppm)
                Магний (Mg): 50 г (500 000 ppm)
                Кальций (Ca): 50 г (500 000 ppm)
            Кислотность (pH): 6.0
            EC/TDS: 1.2 mS/cm
        1:Питательная смесь "Базовая":
            Состав (на 100 литров воды):
                Азот (N): 12 г (120 000 ppm)
                Фосфор (P): 8 г (80 000 ppm)
                Калий (K): 10 г (100 000 ppm)
                Магний (Mg): 3 г (30 000 ppm)
                Кальций (Ca): 3 г (30 000 ppm)
            Кислотность (pH): 6.5
            EC/TDS: 1.3 mS/cm
        2:Питательная смесь "Ростовая":
            Состав (на 100 литров воды):
                Азот (N): 25 г (250 000 ppm)
                Фосфор (P): 5 г (50 000 ppm)
                Калий (K): 15 г (150 000 ppm)
                Магний (Mg): 3 г (30 000 ppm)
                Кальций (Ca): 3 г (30 000 ppm)
            Кислотность (pH): 6.2
            EC/TDS: 1.4 mS/cm
    mixtures_fruiting = 
        0: Питательная смесь "Цветение и плодоношение"
            Состав (на 100 литров воды):
                Азот (N): 10 г (100 000 ppm)
                Фосфор (P): 15 г (150 000 ppm)
                Калий (K): 30 г (300 000 ppm)
                Магний (Mg): 5 г (50 000 ppm)
                Кальций (Ca): 5 г (50 000 ppm)
            Кислотность (pH): 6.5
            EC/TDS: 1.8 mS/cm
        1: Питательная смесь "Ростовая":
                Состав (на 100 литров воды):
                    Азот (N): 25 г (250 000 ppm)
                    Фосфор (P): 5 г (50 000 ppm)
                    Калий (K): 15 г (150 000 ppm)
                    Магний (Mg): 3 г (30 000 ppm)
                    Кальций (Ca): 3 г (30 000 ppm)
                Кислотность (pH): 6.2
                EC/TDS: 1.4 mS/cm
        2: Питательная смесь "Цветение и плодоношение":
            Состав (на 100 литров воды):
                Азот (N): 15 г (150 000 ppm)
                Фосфор (P): 10 г (100 000 ppm)
                Калий (K): 25 г (250 000 ppm)
                Магний (Mg): 5 г (50 000 ppm)
                Кальций (Ca): 5 г (50 000 ppm)
            Кислотность (pH): 6.3
            EC/TDS: 1.5 mS/cm
        
       water:'N':0,'P':0,'K':0,'Mg':0,'Ca':0,'pH':7,'EC':1.8
    '''
```

In [ ]:
env = SoilGreenHouse(stage='seedling',valves_coefs=[0,1])

print([*env.observation_space.shape],*env.action_space.shape)

input_dims = [*env.observation_space.shape]
n_actions = env.action_space.shape[0]

print(input_dims,n_actions)


def make_gif(frame_folder,name):
      frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
      frame_one = frames[0]
      frame_one.save(os.path.join(frame_folder,f"{name}.gif"), format="GIF", append_images=frames,
                save_all=True, duration=100, loop=0)
def render_mp4(videopath: str) -> str:
    """
    Gets a string containing a b4-encoded version of the MP4 video
    at the specified path.
    """
    mp4 = open(videopath, 'rb').read()
    base64_encoded_mp4 = b64encode(mp4).decode()
    return f'<video width=400 controls><source src="data:video/mp4;' \
          f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

In [ ]:
kwargs = {
        "state_dim": input_dims[0],
        "action_dim": n_actions,
        "gamma": .99,
        "hid_shape": (256,128),
        "a_lr": 3e-5,
        "c_lr": 3e-5,
        "batch_size":128,
        "alpha":0.12,
        "adaptive_alpha":True
    }
name_env = 'SoilGreenHouse'

agentSAC = AgentSAC(**kwargs, agent_dir=os.path.join(os.getcwd(),name_env,'SAC'))

if not os.path.exists(os.path.join(os.getcwd(),name_env,'SAC')):
    os.mkdir(os.path.join(os.getcwd(),name_env,'SAC'))
BriefEnvName = ['SGH','BWv3', 'BWHv3', 'Lch_Cv2', 'PV0', 'Humanv2', 'HCv2']

EnvIdex = 0

random_seed = 1

In [ ]:
timenow = str(datetime.now())[0:-10]
timenow = ' ' + timenow[0:13] + '_' + timenow[-2::]
writepath = os.path.join(os.getcwd(),name_env,'SAC','runs',f'SAC_{BriefEnvName[EnvIdex]}' + timenow)
if os.path.exists(writepath):
    shutil.rmtree(writepath)
writer= SummaryWriter(log_dir=writepath)
print(f'writepath: {writepath}')

#### rewrite train

In [ ]:
import types

In [ ]:
env.reset()

In [ ]:
 def train(
        self,env,total_steps,EnvIdex,write,writer,
        EnvName = ['SGH','BWv3', 'BWHv3', 'Lch_Cv2', 'PV0', 'Humanv2', 'HCv2'],
        update_every:int=50,
        eval_interval:int=int(1e3),
        save_interval:int=int(2.5e3),# opt.save_interval  #in steps
        random_seed:int=42,
        plot:bool=True,
        plot_path:str= os.path.join(os.getcwd(),'plots',"BipedalWalker-v3"),
        isProfile:bool=False):


            if not os.path.exists(plot_path):
                    # Create a new directory because it does not exist
                    os.makedirs(plot_path)
            replay_buffer = RandomBuffer([*env.observation_space.shape][0], env.action_space.shape[0], True, max_size=int(1e6))
            max_action = float(env.action_space.high[0])
            steps_per_epoch = env._max_episode_steps
            #Interaction config:

            write = True # opt.write

            start_steps = 5 * steps_per_epoch #in steps
            update_after = 2 * steps_per_epoch #in steps
            eval_env = env


            s, done, current_steps = env.reset(), False, 0


            # print(f's={s}; done = {done};')
            for t in range(total_steps):
                current_steps += 1
                '''Interact & trian'''

                if t < start_steps:
                    #Random explore for start_steps
                    act = env.action_space.sample() #act∈[-max,max]
                    a = Action_adapter_reverse(act,max_action) #a∈[-1,1]
                else:
                    # print(f'[AgentSAC] total_steps = {total_steps}; [Action_adapter];s={s[0]}; t = {t}')
                    a = self.select_action(s, deterministic=False, with_logprob=False) #a∈[-1,1]
                    act = Action_adapter(a,max_action) #act∈[-max,max]

                s_prime, r, done, info = env.step(act,isProfile)#s_prime, r, done, info = env.step(act)
                dead = Done_adapter(r, done, current_steps, EnvIdex)
                r = Reward_adapter(r, EnvIdex)
                replay_buffer.add(s, a, r, s_prime, dead)
                s = s_prime


                # 50 environment steps company with 50 gradient steps.
                # Stabler than 1 environment step company with 1 gradient step.
                if t >= update_after and t % update_every == 0:
                    for j in range(update_every):
                        # print(f'[AgentSAC] [LEARNING: STEP] = {j} from {update_every} steps')
                        self.learn(replay_buffer)

                '''save model'''
                if (t + 1) % save_interval == 0:
                    self.save(t + 1)

                '''record & log'''
                if (t + 1) % eval_interval == 0:
                    score = evaluate_policy(eval_env, self, False, steps_per_epoch, max_action, EnvIdex)
                    if write:
                        writer.add_scalar('ep_r', score, global_step=t + 1)
                        writer.add_scalar('alpha', self.alpha, global_step=t + 1)
                    print('EnvName:', EnvName[EnvIdex], 'seed:', random_seed, 'totalsteps:', t+1, 'score:', score)
                if done:
                    s, done, current_steps = env.reset(), False, 0
                self.score_history.append(r)
                if plot:
                     if (t + 1) % save_interval == 0:
                        filename = f'{t+1}.png'
                        plotLearning(self.score_history,os.path.join(plot_path,filename),window=100)


            env.close()
            eval_env.close()

agentSAC.train = types.MethodType(train, agentSAC)

In [ ]:
def evaluate_policy(env, model, render, steps_per_epoch, max_action, EnvIdex):
    scores = 0
    turns = 3#opt.eval_turn
    for j in range(turns):
        s, done, ep_r = env.reset(), False, 0
        while not done:
            # Take deterministic actions at test time
            # print(f'[evaluate_policy] s= {s}')
            a = model.select_action(s, deterministic=True, with_logprob=False)
            act = Action_adapter(a, max_action)  # [0,1] to [-max,max]
            s_prime, r, done, info = env.step(act,False)
            ep_r += r
            s = s_prime
            if render:
                env.render()
        # print(ep_r)
        scores += ep_r
    return scores/turns

In [ ]:
!pip freeze > requirements.txt

#### train

In [ ]:
%load_ext tensorboard
agentSAC.train(
    env=env,total_steps=int(1e5),EnvIdex = 0,write=True,
    writer= writer,
    plot=True,
    plot_path=os.path.join(os.getcwd(),name_env,'SAC','plots'),
    isProfile=False)